# Multi-Agent Systems with Semantic Kernel

Welcome to this workshop on multi-agent systems using Semantic Kernel! In this notebook, we'll explore how to create, configure, and orchestrate multiple agents to collaborate in solving complex tasks.

## What You'll Learn
- How to transition from single agents to multi-agent systems
- The fundamentals of AgentGroupChat for agent collaboration
- How to design specialized agents with different roles
- Strategies for managing agent conversations and turn-taking
- Techniques for controlling conversation flow and termination

Let's start by setting up our environment.

In [1]:
import os
import asyncio
from dotenv import load_dotenv

# Import Semantic Kernel components
import semantic_kernel as sk
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)
from semantic_kernel.contents import ChatHistory, ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions import KernelArguments
from semantic_kernel.agents.strategies import (
    KernelFunctionSelectionStrategy,
    KernelFunctionTerminationStrategy,
    SequentialSelectionStrategy,
    DefaultTerminationStrategy,
)

from semantic_kernel.agents.strategies.selection.selection_strategy import (
    SelectionStrategy,
)


# Load environment variables from .env file
load_dotenv()

# Display the environment variables for debugging, from .env file
print("Environment Variables:")
for key, value in os.environ.items():
    print(f"{key}: {value}")

print("Environment set up successfully!")

Environment Variables:
ALLUSERSPROFILE: C:\ProgramData
APPDATA: C:\Users\alvanakievi\AppData\Roaming
APPLICATIONINSIGHTS_CONFIGURATION_CONTENT: {}
APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL: 1
CHROME_CRASHPAD_PIPE_NAME: \\.\pipe\crashpad_25408_QYQTWXWSMCZNHTVK
COMMONPROGRAMFILES: C:\Program Files\Common Files
COMMONPROGRAMFILES(X86): C:\Program Files (x86)\Common Files
COMMONPROGRAMW6432: C:\Program Files\Common Files
COMPUTERNAME: EVI
COMSPEC: C:\Windows\system32\cmd.exe
DRIVERDATA: C:\Windows\System32\Drivers\DriverData
EFC_8704: 1
ELECTRON_RUN_AS_NODE: 1
FPS_BROWSER_APP_PROFILE_STRING: Internet Explorer
FPS_BROWSER_USER_PROFILE_STRING: Default
GOPATH: C:\Users\alvanakievi\go
HOMEDRIVE: C:
HOMEPATH: \Users\alvanakievi
JPY_INTERRUPT_EVENT: 5588
LOCALAPPDATA: C:\Users\alvanakievi\AppData\Local
LOGONSERVER: \\EVI
NUMBER_OF_PROCESSORS: 8
ONEDRIVE: C:\Users\alvanakievi\OneDrive - Microsoft
ONEDRIVECOMMERCIAL: C:\Users\alvanakievi\OneDrive - Microsoft
ORIGINAL_XDG_CURRENT_DESKTOP: undefine

Let's create our helper function for setting up a kernel with AI services, similar to what we used in the previous notebook:

In [2]:
def create_kernel_with_service(service_id):
    kernel = sk.Kernel()
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
            deployment_name=os.getenv("AZURE_OPENAI_MODEL_DEPLOYMENT_NAME"),
            api_key=os.getenv('AZURE_OPENAI_API_KEY'),
            endpoint=os.getenv('AZURE_OPENAI_ENDPOINT')
        )
    )
    return kernel

# Create our kernel
kernel = create_kernel_with_service(service_id="multi-agent")
print("Kernel created successfully!")

Kernel created successfully!


In [3]:
# Create our kernel
kernel = create_kernel_with_service(service_id="multi-agent")


# Create the agents with the initialized kernel
agent_factual = ChatCompletionAgent(
    kernel=kernel,
    name="Factual",
    instructions="You are a fact-based agent who provides accurate and concise information. Always stick to verified facts and cite sources when possible. Keep your responses very concise, clear and straightforward.",
)

agent_creative = ChatCompletionAgent(
    kernel=kernel,
    name="Creative",
    instructions="You are a creative agent who thinks outside the box. Offer innovative perspectives and unique ideas. Feel free to brainstorm and suggest creative solutions. Keep your responses very concise, imaginative and engaging.",
)

# Create a simple group chat with these agents
simple_group_chat = AgentGroupChat(
    agents=[agent_factual, agent_creative],
    selection_strategy=SequentialSelectionStrategy(),
    termination_strategy=DefaultTerminationStrategy(maximum_iterations=4),
)

print(f"Created a simple group chat with {len(simple_group_chat.agents)} agents")
print(f"Maximum iterations: {simple_group_chat.termination_strategy.maximum_iterations}")

Created a simple group chat with 2 agents
Maximum iterations: 4


## 1. Introduction to Multi-Agent Systems

### From Single Agents to Agent Systems

While single agents are powerful for many applications, some tasks benefit from having multiple specialized agents working together. Multi-agent systems provide several advantages:

1. **Specialization**: Each agent can focus on what it does best
2. **Role separation**: Different agents can take on different roles (critic, creator, researcher, etc.)
3. **System reliability**: If one agent struggles, others can compensate
4. **Complex problem-solving**: Breaking down problems into sub-tasks handled by different agents
5. **Natural collaboration flow**: Mimicking how humans collaborate in teams

Think of a multi-agent system like a team meeting where each participant has different expertise and contributes to the discussion based on their strengths.

### AgentGroupChat Fundamentals

The `AgentGroupChat` class is the core component for multi-agent collaboration in Semantic Kernel. It provides a framework for multiple agents to interact in a structured conversation.

Key components of an `AgentGroupChat` include:

1. **Agents**: The collection of agents participating in the conversation
2. **Selection Strategy**: Determines which agent speaks next
3. **Termination Strategy**: Decides when the conversation should end
4. **Chat History**: The shared conversation context visible to all agents


The conversation flow is typically:

1. User adds a message to the group chat
2. Selection strategy chooses which agent responds next
3. Selected agent processes the conversation history and generates a response
4. Response is added to the chat history
5. Termination strategy checks if conversation should end
6. If not terminated, process repeats from step 2

Let's start with a simple example: creating two basic agents and connecting them through an AgentGroupChat.

In [4]:
# Create two simple agents with different perspectives
agent_factual = ChatCompletionAgent(
    kernel=kernel,
    name="Factual",
    instructions="You are a fact-based agent who provides accurate and concise information. Always stick to verified facts and cite sources when possible. Keep your responses very concise, clear and straightforward.",
)

agent_creative = ChatCompletionAgent(
    kernel=kernel,
    name="Creative",
    instructions="You are a creative agent who thinks outside the box. Offer innovative perspectives and unique ideas. Feel free to brainstorm and suggest creative solutions. Keep your responses very concise, imaginative and engaging.",
)

# Create a simple group chat with these agents
simple_group_chat = AgentGroupChat(
    agents=[agent_factual, agent_creative],
    # Use sequential selection (round-robin) and limit to 4 total responses
    selection_strategy=SequentialSelectionStrategy(),
    termination_strategy=DefaultTerminationStrategy(maximum_iterations=4),
)

print(f"Created a simple group chat with {len(simple_group_chat.agents)} agents")
print(f"Maximum iterations: {simple_group_chat.termination_strategy.maximum_iterations}")

Created a simple group chat with 2 agents
Maximum iterations: 4


In [5]:
async def run_group_chat(chat, user_message):
    """Run a multi-agent conversation and display the results.

    Args:
        chat: The AgentGroupChat instance
        user_message: The initial user message to start the conversation

    Returns:
        The chat history containing all messages
    """
    # Create a new chat history if needed
    if not hasattr(chat, "history") or chat.history is None:
        # Some versions of AgentGroupChat might not initialize history
        chat_history = ChatHistory()
        chat.history = chat_history

    # Add the user message to the chat
    await chat.add_chat_message(message=user_message)
    print(f"\nUser: {user_message}\n")
    print("=== Beginning Agent Collaboration ===")

    # Track which agent is speaking for formatting
    current_agent = None

    # Invoke the chat and process agent responses
    try:
        async for response in chat.invoke():
            if response is not None and response.name:
                # Add a separator between different agents
                if current_agent != response.name:
                    current_agent = response.name
                    print(f"\n## {response.name}:\n{response.content}")
                else:
                    # Same agent continuing
                    print(f"{response.content}")

        print("\n=== Agent Collaboration Complete ===")
    except Exception as e:
        print(f"Error during chat invocation: {str(e)}")

    # Reset is_complete to allow for further conversations
    chat.is_complete = False

    return chat.history

In [17]:
# Test the simple group chat with a question
question = "What are some approaches to address climate change?"
chat_history = await run_group_chat(simple_group_chat, question)


User: What are some approaches to address climate change?

=== Beginning Agent Collaboration ===

## Factual:
Some effective approaches to address climate change include:

1. **Transition to Renewable Energy**: Shift from fossil fuels to renewable sources like solar, wind, and hydroelectric power.

2. **Energy Efficiency Improvements**: Enhance the efficiency of buildings, appliances, and transportation to lower energy consumption.

3. **Carbon Pricing Mechanisms**: Implement carbon taxes or cap-and-trade systems to incentivize emission reductions.

4. **Afforestation and Reforestation**: Plant trees and restore forests to absorb CO2 from the atmosphere.

5. **Sustainable Transportation**: Promote public transit, electric vehicles, and non-motorized transport options like cycling.

6. **Conservation and Sustainable Land Use**: Protect ecosystems and promote sustainable agriculture to increase carbon sequestration.

7. **Waste Reduction Strategies**: Implement recycling, composting, an

In a multi-agent system, each agent should have a clearly defined role and purpose. Here are some common agent roles you might implement:

| Role Type | Purpose | Example Instructions |
|-----------|---------|---------------------|
| **Researcher** | Provides factual information | Focus on accurate information with sources |
| **Creative** | Generates novel ideas | Think outside conventional boundaries |
| **Critic** | Analyzes and critiques | Identify weaknesses and suggest improvements |
| **Summarizer** | Condenses information | Extract key points concisely |
| **Moderator** | Guides conversation | Keep discussion on track and productive |
| **Domain Expert** | Provides specialized knowledge | Share deep expertise in a specific field |

Let's create a more sophisticated team of agents with specialized roles:

In [19]:
# Create a team of specialized agents

# 1. Researcher who provides factual information
researcher = ChatCompletionAgent(
    kernel=kernel,
    name="Researcher",
    instructions="""You are a thorough researcher who provides factual information and analysis.
    
    Your responsibilities:
    - Provide accurate, factual information on the topic
    - Analyze questions from multiple angles
    - Consider historical context and current understanding
    - Be objective and balanced in your assessment
    - Acknowledge limitations in current knowledge when appropriate
    
    Always strive for accuracy over speculation. Keep your responses very concise, clear and straightforward.""",
)

# 2. Creative thinker who generates innovative ideas
innovator = ChatCompletionAgent(
    kernel=kernel,
    name="Innovator",
    instructions="""You are an innovative thinker who generates novel ideas and perspectives.
    
    Your responsibilities:
    - Suggest unique approaches and solutions
    - Think beyond conventional boundaries
    - Make unexpected connections between concepts
    - Offer imaginative scenarios and possibilities
    - Propose 'what if' scenarios to expand thinking
    
    Don't be constrained by traditional thinking - be bold and creative. Keep your responses very concise, imaginative and engaging.""",
)

# 3. Critic who evaluates ideas critically
critic = ChatCompletionAgent(
    kernel=kernel,
    name="Critic",
    instructions="""You are a thoughtful critic who evaluates ideas and identifies potential issues.
    
    Your responsibilities:
    - Analyze the strengths and weaknesses of proposals
    - Identify potential problems or limitations
    - Challenge assumptions constructively
    - Suggest improvements to ideas
    - Consider practical implementation challenges
    
    Be constructive in your criticism - your goal is to improve ideas, not dismiss them. Keep your responses very concise, clear and straightforward.""",
)

# 4. Synthesizer who brings ideas together
synthesizer = ChatCompletionAgent(
    kernel=kernel,
    name="Synthesizer",
    instructions="""You are a skilled synthesizer who integrates diverse perspectives into coherent conclusions.
    
    Your responsibilities:
    - Identify common themes across different viewpoints
    - Reconcile apparently conflicting ideas when possible
    - Create a balanced, integrated perspective
    - Summarize key points from the discussion
    - Draw reasonable conclusions from the collective input
    
    Your goal is to bring together different perspectives into a coherent whole. Keep your responses very concise, clear and straightforward.""",
)

# Create a group chat with all specialized agents
expert_team = AgentGroupChat(
    agents=[researcher, innovator, critic, synthesizer],
    selection_strategy=SequentialSelectionStrategy(),
    termination_strategy=DefaultTerminationStrategy(
        maximum_iterations=8
    ),  # 2 rounds of all 4 agents
)

print(f"Created an expert team with {len(expert_team.agents)} specialized agents")

Created an expert team with 4 specialized agents


Now let's test our expert team with the same climate change question to see how their specialized roles affect the collaboration:

In [10]:
chat_history = await run_group_chat(expert_team, question)


User: What are some approaches to address climate change?

=== Beginning Agent Collaboration ===

## Researcher:
To effectively address climate change, various approaches can be taken, categorized into three main areas: mitigation, adaptation, and policy frameworks. Each of these areas encompasses multiple strategies.

### Mitigation Strategies
1. **Renewable Energy**: Transitioning from fossil fuels to renewable sources such as solar, wind, and hydroelectric power to reduce greenhouse gas emissions.
2. **Energy Efficiency**: Implementing technologies and practices that enhance energy efficiency in homes, buildings, and industries to lower energy consumption.
3. **Carbon Sequestration**: Developing carbon capture and storage technologies to remove CO2 from the atmosphere and store it underground or repurpose it.
4. **Sustainable Transportation**: Promoting public transit, electric vehicles, and cycling infrastructure to decrease reliance on fossil fuel-powered transportation.
5. **Sus

CancelledError: 

Compare the two approaches:

1. **Simple Group Chat (2 agents)**:
   - Basic factual and creative perspectives
   - Limited interaction depth
   - Simple back-and-forth exchange

2. **Expert Team (4 specialized agents)**:
   - Multiple specialized perspectives
   - More comprehensive analysis
   - Natural progression from facts → ideas → critique → synthesis

This demonstrates how designing agents with complementary roles can create more valuable interactions and outputs.

## 2. Introduction Agent Strategies

In the previous section, we created basic multi-agent systems using `AgentGroupChat` with default strategies. Now let's explore how to build more sophisticated agent collaborations by configuring agents with specialized roles and customizing their interactions.

### Initializing Multiple Specialized Agents

When building effective multi-agent systems, the way you design and initialize your agents is critical. Each agent should have:

1. **Clear purpose**: A well-defined role and responsibility
2. **Specialized instructions**: Guidance on how to approach problems from its unique perspective
3. **Complementary skills**: Abilities that work well with other agents' capabilities
4. **Communication style**: How it should present information to other agents

Let's create a team of specialized agents for a specific task: collaborative writing and editing.

In [11]:
# Create a writer agent that generates content
writer = ChatCompletionAgent(
    kernel=kernel,
    name="Writer",
    instructions="""You are a creative writer who crafts engaging content.
    
    Your role in this collaboration:
    1. Generate original content based on the topic provided
    2. Apply creative storytelling techniques to engage readers
    3. Incorporate feedback from the editor and fact-checker to improve your writing
    4. Revise content to address issues raised by other team members
    5. Focus on creating a compelling narrative voice and structure
    
    When responding to feedback:
    - Be open to constructive criticism
    - Explain your creative choices when relevant
    - Incorporate suggestions that improve the content
    
    Always strive to maintain the core message while making the content more engaging and effective. Keep your responses very concise, imaginative and engaging.""",
)

# Create an editor agent that improves the writing
editor = ChatCompletionAgent(
    kernel=kernel,
    name="Editor",
    instructions="""You are a meticulous editor who improves content quality and clarity.
    
    Your role in this collaboration:
    1. Review content for clarity, coherence, and flow
    2. Identify and fix grammatical, structural, or stylistic issues
    3. Suggest improvements to enhance readability and impact
    4. Ensure the content meets its intended purpose and audience needs
    5. Maintain consistent voice and tone throughout
    
    When providing feedback:
    - Be specific about what needs improvement and why
    - Offer constructive suggestions rather than just criticism
    - Consider both micro (sentence-level) and macro (structure) improvements
    - Balance preserving the writer's voice with improving the content
    
    Your goal is to elevate the writing while respecting the writer's intent and style. Keep your responses very concise, clear and straightforward.""",
)

# Create a fact-checker agent that ensures accuracy
fact_checker = ChatCompletionAgent(
    kernel=kernel,
    name="FactChecker",
    instructions="""You are a thorough fact-checker who ensures accuracy and credibility.
    
    Your role in this collaboration:
    1. Verify factual claims in the content
    2. Identify potential inaccuracies or misleading statements
    3. Suggest corrections for any factual errors
    4. Recommend additional context where needed for accuracy
    5. Ensure the content is truthful and well-supported
    
    When providing feedback:
    - Focus on accuracy rather than style or structure
    - Explain why a statement might be problematic
    - Provide correct information to replace inaccuracies
    - Consider potential sources of factual support
    
    Your goal is to ensure the content maintains high standards of accuracy and integrity. Keep your responses very concise, clear and straightforward.""",
)

print("Created specialized writing team agents successfully!")

Created specialized writing team agents successfully!


### Setting Up a Basic Group Chat

Now that we have our specialized agents, let's set up an `AgentGroupChat` to orchestrate their collaboration. We need to consider:

1. **Agent order**: The sequence in which agents participate
2. **Maximum iterations**: How many turns the conversation should take
3. **Turn-taking logic**: How to determine which agent speaks next

For this example, we'll use a simple sequential approach where agents take turns in a fixed order.

In [12]:
# Create a writing team chat with sequential selection strategy
writing_team = AgentGroupChat(
    agents=[writer, editor, fact_checker],
    # Specify that the writer should start
    selection_strategy=SequentialSelectionStrategy(initial_agent=writer),
    # Limit to 9 turns total (3 rounds × 3 agents)
    termination_strategy=DefaultTerminationStrategy(maximum_iterations=9),
)

print("Writing team chat created with sequential selection strategy")
print(f"Initial agent: {writing_team.selection_strategy.initial_agent.name}")
print(f"Maximum iterations: {writing_team.termination_strategy.maximum_iterations}")

Writing team chat created with sequential selection strategy
Initial agent: Writer
Maximum iterations: 9


The `SequentialSelectionStrategy` we're using has these properties:

- It follows a round-robin pattern, moving through agents in the order they were added
- You can specify an `initial_agent` to start the conversation
- It's predictable and ensures each agent gets a turn to contribute

Now let's test our writing team collaboration with a writing task:

In [13]:
# Test the writing team with a specific writing task
writing_task = "Write a short blog post about the impact of artificial intelligence on healthcare, focusing on recent advancements and ethical considerations."

# Run the group chat
chat_history = await run_group_chat(writing_team, writing_task)


User: Write a short blog post about the impact of artificial intelligence on healthcare, focusing on recent advancements and ethical considerations.

=== Beginning Agent Collaboration ===

## Writer:
**Transforming Healthcare: The Dual Edge of Artificial Intelligence**

In recent years, artificial intelligence (AI) has emerged as a groundbreaking force in healthcare, reshaping the landscape of diagnosis, treatment, and patient care. Picture a world where your doctor, aided by algorithms, predicts illnesses even before symptoms manifest. This synergy is becoming reality, with AI systems analyzing vast datasets to identify patterns invisible to the human eye. For instance, machine learning models have shown remarkable accuracy in detecting diseases from medical imaging, often outperforming traditional diagnostic methods.

However, with great power comes great responsibility—enter the ethical considerations. The promise of AI in healthcare isn't free of pitfalls. The reliance on AI algor

Did you notice the pattern in the conversation?

1. The **Writer** created the initial draft
2. The **Editor** reviewed it for clarity and style
3. The **FactChecker** verified the factual claims
4. The **Writer** revised based on feedback
5. The **Editor** reviewed the revised version
6. And so on...

This sequential approach mimics a real editorial process where content moves through different stages of review and revision.

### Exercise: Create Your First Multi-Agent Conversation

Now it's your turn to design a multi-agent system! 

In this exercise, you'll create your own team of specialized agents. These agents will collaborate to plan a vacation Follow the steps below to design your multi-agent system.
1. Create a travel planning team with 3 specialized agents (eg Travel Advisor, Budget Planner, Safety Expert)
2. Set up your AgentGroupChat
3. Define a travel planning problem
4. Run the group chat with your travel question

In [ ]:

# Step 1: Create a travel planning team with specialized agents
# Create a Travel Advisor agent

# Create a Budget Planner agent

# Create a Safety Expert agent

# Step 2: Set up your AgentGroupChat
travel_team=


print(f"Created a travel planning team with {len(travel_team.agents)} specialized agents")
print(f"Initial agent: {travel_team.selection_strategy.initial_agent.name}")

# Step 3: Define a travel planning problem
travel_question = "I'm planning a 10-day vacation in June. Can you suggest a good destination and help me plan the trip?"

# Step 4: Run the group chat with your travel question


Created a travel planning team with 3 specialized agents
Initial agent: TravelAdvisor

User: I'm planning a 10-day vacation in June. Can you suggest a good destination and help me plan the trip?

=== Beginning Agent Collaboration ===

## TravelAdvisor:
Absolutely! How about a captivating journey through the breathtaking landscapes of New Zealand’s South Island? It’s a treasure trove of stunning scenery and thrilling adventures waiting to be explored!

**Day 1-2: Christchurch**  
Start your adventure in the vibrant city of Christchurch, known for its beautiful gardens and the historic River Avon. Explore the rejuvenated city center and don't miss the stunning Botanic Gardens. A ride on the Christchurch Tram is a delightful way to soak up the sights while learning about the city’s history.

**Day 3-4: Queenstown**  
Next, head to the adventure capital of the world—Queenstown! Nestled beside the sparkling Lake Wakatipu and surrounded by the towering Southern Alps, this town is packed with

<details>
<summary>Click to see solution</summary>

```python
# Create a Travel Advisor agent
travel_advisor = ChatCompletionAgent(
    kernel=kernel,
    name="TravelAdvisor",
    instructions="""You are a knowledgeable travel advisor who recommends destinations and attractions.
    
    Focus only on suggesting interesting places to visit and activities to do.
    Do not discuss budgets, safety issues, or logistics - other agents will handle those.
    
    Write in an enthusiastic, descriptive style that brings destinations to life.""",
)

# Create a Budget Planner agent
budget_planner = ChatCompletionAgent(
    kernel=kernel,
    name="BudgetPlanner",
    instructions="""You are a practical budget planner focused exclusively on travel costs.
    
    Focus only on estimating expenses, suggesting money-saving tips, and financial considerations.
    Do not recommend destinations or discuss safety issues - other agents handle those.
    
    Use specific dollar amounts and practical money-saving advice in your responses.""",
)

# Create a Safety Expert agent
safety_expert = ChatCompletionAgent(
    kernel=kernel,
    name="SafetyExpert",
    instructions="""You are a travel safety expert who identifies potential risks and precautions.
    
    Focus only on health considerations, safety tips, and risk assessments for travelers.
    Do not recommend destinations or discuss costs - other agents handle those.
    
    Be factual and practical in your safety recommendations.""",
)

# Step 2: Set up your AgentGroupChat
travel_team = AgentGroupChat(
    agents=[travel_advisor, budget_planner, safety_expert],
    selection_strategy=SequentialSelectionStrategy(initial_agent=travel_advisor),
    termination_strategy=DefaultTerminationStrategy(maximum_iterations=6),
)

print(f"Created a travel planning team with {len(travel_team.agents)} specialized agents")
print(f"Initial agent: {travel_team.selection_strategy.initial_agent.name}")

# Step 3: Define a travel planning problem
travel_question = "I'm planning a 10-day vacation in June. Can you suggest a good destination and help me plan the trip?"

# Step 4: Run the group chat with your travel question
chat_history = await run_group_chat(travel_team, travel_question)
```
</details>

### Simple Agent Collaboration Patterns

The sequential selection we just used is one of several agent collaboration patterns. Here are some common patterns you might use:

1. **Sequential (Round-Robin)**: Agents take turns in a fixed order
   - Pros: Simple, predictable, ensures each agent participates
   - Cons: Not responsive to content; might include irrelevant contributions

2. **Fixed Workflow**: Agents follow a specific sequence designed for a task
   - Pros: Mirrors real workflows (e.g., write → edit → approve)
   - Cons: Less flexible for unexpected situations

3. **Dynamic Selection**: An LLM decides which agent should respond next
   - Pros: More responsive to conversation needs; adaptable
   - Cons: Less predictable; requires careful prompt engineering

Let's implement a fixed workflow pattern for our writing team, where we explicitly define the sequence of agents based on the writing process:

In [15]:
from typing import List

# Custom function to create a fixed workflow chat
def create_fixed_workflow_chat(agents, workflow_sequence, max_iterations=None):
    """Create a chat with a fixed agent workflow sequence.

    Args:
        agents: List of agent instances
        workflow_sequence: List of agent names in the desired workflow order
        max_iterations: Maximum number of iterations (default: len(workflow_sequence))
    """
    # Create a mapping of agent names to instances
    agent_map = {agent.name: agent for agent in agents}

    # Verify all workflow agents exist
    for name in workflow_sequence:
        if name not in agent_map:
            raise ValueError(f"Agent '{name}' in workflow not found in provided agents")

    # Create a custom selection strategy class
    class FixedWorkflowStrategy(SelectionStrategy):
        counter: int = 0
        workflow_sequence: List[str]

        async def next(self, agents, messages):
            print("workflow_sequence", self.workflow_sequence)
            agent = agent_map[
                self.workflow_sequence[self.counter % len(self.workflow_sequence)]
            ]
            self.counter += 1
            print("selected:", agent.name)

            self.has_selected = True
            return agent

    # Set maximum iterations if not specified
    if max_iterations is None:
        max_iterations = len(workflow_sequence)

    fixed_workflow_strategy = FixedWorkflowStrategy(workflow_sequence=workflow_sequence)

    # Create and return the AgentGroupChat
    return AgentGroupChat(
        agents=agents,
        selection_strategy=fixed_workflow_strategy,
        termination_strategy=DefaultTerminationStrategy(
            maximum_iterations=max_iterations
        ),
    )


# Define a specific writing workflow
# Writer -> Editor -> FactChecker -> Writer (for revisions) -> Editor (final review)
writing_workflow = ["Writer", "Editor", "FactChecker", "Writer", "Editor"]

# Create the workflow chat
workflow_chat = create_fixed_workflow_chat(
    agents=[writer, editor, fact_checker],
    workflow_sequence=writing_workflow,
    max_iterations=len(writing_workflow),
)

print(f"Created fixed workflow chat with sequence: {' → '.join(writing_workflow)}")

Created fixed workflow chat with sequence: Writer → Editor → FactChecker → Writer → Editor


Now let's test our fixed workflow collaboration with the same writing task:

In [16]:
# Test the workflow chat with the same writing task
chat_history = await run_group_chat(workflow_chat, writing_task)


User: Write a short blog post about the impact of artificial intelligence on healthcare, focusing on recent advancements and ethical considerations.

=== Beginning Agent Collaboration ===
workflow_sequence ['Writer', 'Editor', 'FactChecker', 'Writer', 'Editor']
selected: Writer

## Writer:
**Title: The Transformative Touch of AI in Healthcare: Innovations and Ethical Dilemmas**

In recent years, artificial intelligence (AI) has emerged as a beacon of innovation in healthcare, revolutionizing everything from diagnostics to treatment personalization. Imagine a world where diseases are detected with uncanny precision, allowing doctors to intervene before symptoms manifest. This is not merely a dream; it is our new reality.

Recent advancements in machine learning algorithms have empowered AI to analyze vast datasets, discovering patterns that even the most skilled human eyes may overlook. For instance, AI-driven imaging tools can identify early-stage cancers in medical imaging scans with

The fixed workflow gives us more control over the exact sequence of agent interactions. This is especially useful for process-oriented tasks that have a natural sequence, like content creation and review.

Notice how we customized the workflow to match a realistic content creation process:
1. **Writer**: Creates initial draft
2. **Editor**: Reviews for style and clarity
3. **FactChecker**: Verifies accuracy
4. **Writer**: Revises based on all feedback
5. **Editor**: Gives final approval

This is more directed than the simple round-robin approach and can lead to a more efficient collaboration for specific tasks.